In [1]:
import numpy as np

#import mixture
import import_ipynb
import mixture
import util



def multinomial_entropy(p):
    """Compute the entropy of a Bernoulli random variable, in nats rather than bits."""
    p = np.clip(p, 1e-20, np.infty)      # avoid taking the log of 0
    return -np.sum(p * np.log(p))


def variational_objective(model, X, R, pi, theta):
    """Compute the variational lower bound on the log-likelihood that each step of E-M
    is maximizing. This is described in the paper

        Neal and Hinton, 1998. A view of the E-M algorithm that justifies incremental, sparse, and other variants.

    We can test the update rules by verifying that each step maximizes this bound.
    """

    model = mixture.Model(model.prior, mixture.Params(pi, theta))
    expected_log_prob = model.expected_joint_log_probability(X, R)
    entropy_term = np.sum(multinomial_entropy(R))
    return expected_log_prob + entropy_term

def perturb_pi(pi, eps=1e-6):
    pi = np.random.normal(pi, eps)
    pi = np.clip(pi, 1e-10, np.infty)
    pi /= pi.sum()
    return pi

def perturb_theta(theta, eps=1e-6):
    theta = np.random.normal(theta, eps)
    theta = np.clip(theta, 1e-10, 1. - 1e-10)
    return theta

def perturb_R(R, eps=1e-6):
    R = np.random.normal(R, eps)
    R = np.clip(R, 1e-10, np.infty)
    R /= R.sum(1).reshape((-1, 1))
    return R



def check_m_step():
    """Check that the M-step updates by making sure they maximize the variational
    objective with respect to the model parameters."""
    np.random.seed(0)

    NUM_IMAGES = 100

    X = util.read_mnist_images(mixture.TRAIN_IMAGES_FILE)
    X = X[:NUM_IMAGES, :]
    R = np.random.uniform(size=(NUM_IMAGES, 10))
    R /= R.sum(1).reshape((-1, 1))

    model = mixture.Model.random_initialization(mixture.Prior.default_prior(), 10, 784)
    
    theta = model.update_theta(X, R)
    pi = model.update_pi(R)
    

    opt = variational_objective(model, X, R, pi, theta)
    
    ok = True
    for i in range(20):
        new_theta = perturb_theta(theta)
        new_obj = variational_objective(model, X, R, pi, new_theta)
        if new_obj > opt:
            ok = False
    if ok:
        print('The theta update seems OK.')
    else:
        print('Something seems to be wrong with the theta update.')

    if not np.allclose(np.sum(pi), 1.):
        print('Uh-oh. pi does not seem to sum to 1.')
    else:
        ok = True
        for i in range(20):
            new_pi = perturb_pi(pi)
            new_obj = variational_objective(model, X, R, new_pi, theta)
            if new_obj > opt:
                ok = False
        if ok:
            print('The pi update seems OK.')
        else:
            print('Something seems to be wrong with the pi update.')


def check_e_step():
    """Check the E-step updates by making sure they maximize the variational
    objective with respect to the responsibilities. Note that this does not
    fully check your solution to Part 2, since it only applies to fully observed
    images."""
    
    np.random.seed(0)

    NUM_IMAGES = 100

    X = util.read_mnist_images(mixture.TRAIN_IMAGES_FILE)
    X = X[:NUM_IMAGES, :]
    model = mixture.train_from_labels(show=False)

    # reduce the number of observations so that the posterior is less peaked
    X = X[:, ::50]
    model.params.theta = model.params.theta[:, ::50]
    
    R = model.compute_posterior(X)

    opt = variational_objective(model, X, R, model.params.pi, model.params.theta)

    if not np.allclose(R.sum(1), 1.):
        print('Uh-oh. Rows of R do not seem to sum to 1.')
    else:
        ok = True
        for i in range(20):
            new_R = perturb_R(R)
            new_obj = variational_objective(model, X, new_R, model.params.pi, model.params.theta)
            if new_obj > opt:
                ok = False
        if ok:
            print('The E-step seems OK.')
        else:
            print('Something seems to be wrong with the E-step.')

if __name__ == '__main__':
    check_e_step()
    check_m_step()






importing Jupyter notebook from mixture.ipynb
pi[0] [0.085]
pi[1] [0.13]
theta[0, 239] 0.6427106227106232
theta[3, 298] 0.46573612495845823
Training log-likelihood: -172.07854196938325
Test log-likelihood: -170.97538133747267
R[0, 2] 0.1748895149211724
R[1, 0] 0.688537676109229
P[0, 183] 0.6516151998131035
P[2, 628] 0.4740801724913304


c:\users\eodus\anaconda3\envs\tensorflow\lib\site-packages\matplotlib\axes\_base.py:3012: UserWarning: Attempting to set identical left==right results
in singular transformations; automatically expanding.
left=1.0, right=1.0
  self.set_xlim(upper, lower, auto=None)


Final training log-likelihood: -138.06314474092903
Final test log-likelihood: -138.54618064790586
Train set case:  (array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]), array([5923, 6742, 5958, 6131, 5842, 5421, 5918, 6265, 5851, 5949],
      dtype=int64))
Test set case:  (array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]), array([ 980, 1135, 1032, 1010,  982,  892,  958, 1028,  974, 1009],
      dtype=int64))
Training set
Average log-probability of a 0 image: -201.136
Average log-probability of a 1 image: -97.755
Average log-probability of a 2 image: -201.701
Average log-probability of a 3 image: -184.880
Average log-probability of a 4 image: -172.018
Average log-probability of a 5 image: -191.928
Average log-probability of a 6 image: -177.253
Average log-probability of a 7 image: -156.895
Average log-probability of a 8 image: -187.626
Average log-probability of a 9 image: -162.041

Test set
Average log-probability of a 0 image: -199.743
Average log-probability of a 1 image: -96.973
Average log-probability of a